#Integrating OpenAI LLM with ChromaDB and Sentence Embedding Models

Integrating Large Language Model (LLM) with ChromaDB and a sentence embedding model using OpenAI's models. Leverage LangChain to build a retrieval-augmented generation (RAG) pipeline to efficiently retrieve relevant data and generate responses.

##Summary
This notebook demonstrates a Retrieval-Augmented Generation (RAG) pipeline that combines OpenAI’s chat‑based LLM with a local HuggingFace fallback model and ChromaDB for vector storage.
* Install and import the required packages (langchain, chromadb, sentence-transformers, etc.)
* Configure and instantiate an OpenAI-based chat model with API‑key authentication
* Build a lightweight local fallback using HuggingFace’s gpt2 pipeline
* Embed and index text data (from a CSV) into ChromaDB using MiniLM embeddings
* Split large documents into manageable chunks for retrieval
* Construct dual RetrievalQA chains (primary via OpenAI, secondary via local HF)
* Execute queries against the indexed data with automated fallback on rate limits or errors

##Prerequisites
Ensure you have the following installed:

In [ ]:
!pip install langchain chromadb sentence-transformers pandas tqdm


In [ ]:
!pip install langchain_openai

In [ ]:
!pip install langchain_community

In [ ]:
!pip install transformers

In [ ]:
!pip install --upgrade openai

## Step 1: Load Necessary Libraries

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
import os
from langchain_openai import ChatOpenAI

In [ ]:
from openai import OpenAIError, RateLimitError

##Step 2: Read OpenAI API Key and Instantiate LLM

In [ ]:
# Retrieve your OpenAI API key from the environment variable "OPENAI_API_KEY"
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key is None:
    # Alternatively, you can set it directly here (not recommended for production)
    openai_api_key = "YOUR_API_KEY_HERE"

In [ ]:
# Instantiate the ChatOpenAI LLM with your API key
llm_openai = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name="gpt-3.5-turbo",
    temperature=0,
)

##Step 2a: Instantiate local HuggingFace LLM text‑generation pipeline

In [ ]:
hf_pipe = pipeline(
    "text-generation",
    model="gpt2",            # lightweight, no API key needed
    max_new_tokens=256, # Increased max_length to accommodate longer inputs
    do_sample=False
)
llm_fallback = HuggingFacePipeline(pipeline=hf_pipe)

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


##Step 3: Load Langchain with LLM for RetreivalQA

In [ ]:
chromadb_path = './'
# Use a valid pre-trained model name instead of a local path
embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
vectordb = Chroma(persist_directory=chromadb_path, embedding_function=embedding_model)

/tmp/ipython-input-16-1841999009.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
/tmp/ipython-input-16-1841999009.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=chromadb_path, embedding_function=embedding_model)


In [ ]:
# Create a retriever from the vector database
retriever = vectordb.as_retriever(search_kwargs={"k": 1})

## Step 4a: Load and Process Data

In [ ]:
# 1. Load your CSV(s)
loader = CSVLoader(file_path="CISA_combo_features_new.csv", encoding="utf-8")
raw_docs = loader.load()

# 2. (Optional) Split into smaller chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(raw_docs)

## Step 4b: Add Documents to ChromaDB

In [ ]:
# Add documents into  ChromaDB instance
vectordb.add_documents(docs)

# Persist to disk so you can re‑load later
vectordb.persist()

# Display the number of documents in the vector store
print(f"Number of documents in vector store: {vectordb._collection.count()}")

Number of documents in vector store: 3326


/tmp/ipython-input-19-3238707526.py:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


##Step 5a: Build RetrievalQA Chain using OpenAI model

In [ ]:
# Build the RetrievalQA chain using the OpenAI model
qa_openai = RetrievalQA.from_chain_type(
    llm=llm_openai,
    chain_type="refine",
    retriever=retriever,
    return_source_documents=True,
    verbose=False,
)

##Step 5b: Build a Backup RetrievalQA chain pointing at the same retriever

In [ ]:
qa_fallback = RetrievalQA.from_chain_type(
    llm=llm_fallback,
    chain_type="refine",
    retriever=retriever,
    return_source_documents=True,
    verbose=False,
)

##Step 6: Querying the System

In [ ]:
query = "What is the MITRE ATT&CK technique for remote desktop attacks?"

In [35]:
try:
    response = qa_openai.invoke(query)
    # Print only the main result (you can also inspect source_documents if needed)
    print("✅ Response from OpenAI LLM:", response["result"])
except (RateLimitError, OpenAIError) as e:
    print(f"⚠️ OpenAI LLM failed ({e.__class__.__name__}): {e}")
    print("→ Falling back to local HuggingFace LLM")
    response = qa_fallback.invoke(query, max_new_tokens=256) # Pass max_new_tokens here
    print("✅ Fallback HF response:", response["result"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


⚠️ OpenAI LLM failed (RateLimitError): Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
→ Falling back to local HuggingFace LLM
✅ Fallback HF response: Context information is below. 
------------
Sharing and Analysis Center (MS-ISAC), and the Israel National Cyber Directorate (INCD) released the Guide to Securing Remote Access Software. This new joint guide is the result of a collaborative effort to provide an overview of legitimate uses of remote access software, as well as common exploitations and associated tactics, techniques, and procedures (TTPs), and how to detect and defend against malicious actors abusing this software. Remote access software provides organizations with a broad
------------
Given the context information

##Step 7: Scaling Considerations
When scaling your solution, consider the following factors:

* Storage Needs: Plan according to the volume of data in ChromaDB.
* Performance: Use GPU-based instances for LLM operations.
* Cost Management: Monitor infrastructure costs such as AWS GPU instances.

##Conclusion
By integrating ChromaDB with LLMs, cybersecurity analysts can quickly map alerts to known techniques.
Integrated an LLM with ChromaDB and a sentence embedding model. The setup allows us to harness RAG to improve cybersecurity operations by retrieving relevant data and generating insightful responses.
Benefits:
* High accuracy & flexibility: Leverages state‑of‑the‑art OpenAI models for best‑in‑class responses while retaining a free, local LLM fallback.
* Speed & cost control: Local inference reduces latency and avoids API charges when possible.
* Scalability: ChromaDB’s persist‑and‑query architecture allows you to handle large data volumes and scale out with GPUs or managed services.
* Extensibility: Easily swap in different embedding models, vector stores, or LLMs to suit your domain.